# 1. Time series gene regulatory network inference
We want to infer the gene regulatory network (denote with adjacency matrix $\mathbf{A}$) given the time-series gene expression data $x$. The gene expression data records the change of $10$ gene expression values from time point $1$ to $50$. 

In the lecture, we talked about multiple ways to infer the gene regulatory network with data driven methods, one of the most straightforward ways is to use the differential equation model. Let's assume the gene expression data follows:

$$
\frac{d\mathbf{x}(t)}{dt} = \mathbf{Ax}(t) + \mathbf{n},\, \mathbf{n}\sim \mathcal{N}(\mathbf{0},\mathbf{I})
$$

We can estimate the changing speed of gene expression data $\frac{d\mathbf{x}(t)}{dt}$ at time point $t$($t = 1,2,3,\cdots,49$) with $\Delta \mathbf{x}(t) = \mathbf{x}(t+1) - \mathbf{x}(t)$. And the problem turns into a [least square](https://en.wikipedia.org/wiki/Least_squares#Lasso_method) problem

$$
\mathbf{A} = \arg\min_{\mathbf{A}}\sum_{t = 1}^{49}\Vert\Delta \mathbf{x}(t) - \mathbf{Ax}(t)\Vert_2^2 
$$

Please find the solution of the least square problem above using python. 

(Hint: You need to
* Calculate $\Delta \mathbf{x}(t)$ using $\mathbf{x}(t)$
* Calculate optimal $\mathbf{A}$ using [gradient descent algorithm](https://en.wikipedia.org/wiki/Gradient_descent#:~:text=Gradient%20descent%20is%20a%20first,the%20direction%20of%20steepest%20descent), or by setting the gradient of the objective function(w.r.t $\mathbf{A}$) to be $0$. 

)


In [ ]:
import numpy as np 

In [ ]:
# xs stores the time series data, it is of the dimension (10,50), correspond to the expression data of 10 genes across 50 time points
xs = np.load("counts.npy")
# TODO: implement your code here
tps=xs.shape[1]-1
genes=xs.shape[0]
A= np.random.randint(-1,1,size=(genes,genes))

#Calculate dx(t), where t is the col/timepoint
dx=np.empty((genes, tps))
for tp in range(0,tps):
  dx[:,tp]=xs[:,tp+1]-xs[:,tp]

X=xs[:,0:tps]
Y=dx

iter=300
lr=0.0001

def gradient(X,Y,A):
  return(2*np.dot(np.dot(A,X),X.T)-2*np.dot(Y,X.T))

for it in range(0,iter):
  cost=np.sum(np.dot(A,X)**2)
  A=A-lr*gradient(X,Y,A)
  #print(cost)
A1=A

# 2. Incorporate l1 term 
Gene regulatory network tend to be a sparse graph(have a very small number of $1$s), which can be learned with an $\ell 1$ regularization term. By adding the $\ell 1$ regularization term into the objective function above, the problem turns into a LASSO regression problem:

$$
\mathbf{A} = \arg\min_{\mathbf{A}}\sum_{t = 1}^{49}\Vert\Delta \mathbf{x}(t) - \mathbf{Ax}(t)\Vert_2^2 + \lambda * \Vert\mathbf{A}\Vert_1
$$

$$
\Vert\mathbf{A}\Vert_1 = \sum_i\sum_j| A_{ij}|_1
$$

It is hard to solve a lasso regression problem using traditional gradient descent algorithm, since $\ell 1$ regularization is non-differentiable. **Iterative soft thresholding algorithm** (**ISTA**, a proximal gradient descent algorithm) is a good choice for solving such problem. Please implement ISTA algorithm in the block below.

* Information about ISTA algorithm: https://www.stat.cmu.edu/~ryantibs/convexopt-F15/lectures/08-prox-grad.pdf


In [ ]:
xs = np.load("counts.npy")
# TODO: implement your code here
tps=xs.shape[1]-1
genes=xs.shape[0]
A= np.random.randint(-1,1,size=(genes,genes))

#Calculate dx(t), where t is the col/timepoint
dx=np.empty((genes, tps))
for tp in range(0,tps):
  dx[:,tp]=xs[:,tp+1]-xs[:,tp]

X=xs[:,0:tps]
Y=dx

iter=200
lr=1e-5
lamda=0.01

def gradient(X,Y,A):
  return(2*np.dot(np.dot(A,X),X.T)- 2*np.dot(Y,X.T) - lamda*np.absolute(A1).sum())

for it in range(0,iter):
  cost=np.sum(np.dot(A,X)**2)
  A=A-lr*gradient(X,Y,A)
  #print(cost)
A2=A



# 3. Benchmark
Calculate the difference between the ground truth gene regulatory network(denote as `A`) and the inferred gene regulatory network (step 1 result denoted as `A1`, step 2 result denoted as `A2`) using the `diff` function below. Adjust the hyper-parameter within the algorithms in step $1$ and $2$, and report the best result you can obtain(The smaller the difference, the better the result is).

In [ ]:
A = np.load("gt_grn.npy")

def diff(A_inf, A):
    A_inf = (A_inf > 0.5).astype(np.int)
    return np.sum(np.abs(A_inf - A))/(A.shape[0] * A.shape[1])
    

In [ ]:
# step1
#the Diff stabilizes for lr < 0.001 and iter > 200
diff(A1, A)

0.3

In [ ]:
# step2
diff(A2, A)

0.23